In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [2]:

def get_ma_so_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')
    part1_part_1  = next((i for i, item in enumerate(part1) if "1. Hàng hóa " in item.get_text(strip=True)), -1)
    part1_part_1_text  = part1[part1_part_1].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_2 = next((i for i, item in enumerate(part1) if "Tên thương mại" in item.get_text(strip=True)), -1)
    part1_part_2_text  = part1[part1_part_2].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_3 = next((i for i, item in enumerate(part1) if "Tên gọi theo cấu tạo" in item.get_text(strip=True)), -1)
    part1_part_3_text  = part1[part1_part_3].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_4 = next((i for i, item in enumerate(part1) if "mã hiệu" in item.get_text(strip=True)), -1)
    part1_part_4_text  = part1[part1_part_4].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_5 = next((i for i, item in enumerate(part1) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
    part1_part_5_text  = part1[part1_part_5].get_text().replace('\r', '').replace('\n', '').strip()
    info_list.append(part1_part_2_text)
    info_list.append(part1_part_3_text)
    info_list.append(part1_part_4_text)
    info_list.append(part1_part_5_text)

    # allpart
    table_temp = tables[1]
    for table in tables[1].find_all('table'):
        # Kiểm tra phần tử liền trước của thẻ <table>
        prev_sibling = table.find_previous_sibling()
        if prev_sibling and prev_sibling.name == 'p':
            prev_sibling.decompose()  # Xóa thẻ <p> liền trước
        table.decompose()  # Xóa thẻ <table>
    allpart = table_temp.find_all('p')
    
    # print("allpart: ",allpart)

    
    # lấy part 3
    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in allpart:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        text_2 = p_tag.get_text()
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part3 = matches[-1]
            part3_text = text_2.replace('\r', '').replace('\n', '').strip()
    # lấy part 2
    try :
        part2_start = next((i for i, item in enumerate(allpart) if re.search(r"2\s*\.\s*Tóm tắt", item.get_text())), -1)-1
        # print(part2_start)
    except:
        part2_start = next((i for i, item in enumerate(allpart) if "Nhà sản xuất" in item.get_text()), -1)
        # print(part2_start)
    part2_end = next((i for i, item in enumerate(allpart) if re.search(r"3\s*\.\s*Kết quả", item.get_text(strip=True))), -1)
    part2_list = []
    for i in allpart[part2_start+1:part2_end]:
        part2_list.append(i.get_text().replace('\r', '').replace('\n', '').strip())
    part2_list
    part2_text = '\n'.join(part2_list)
    
    # lấy part 6
    p_before_table1 = []  # Danh sách để lưu trữ thẻ <p>

    for sibling in tables[1].previous_siblings:
        if sibling.name == 'p':
            p_before_table1.append(sibling)
            if len(p_before_table1) == 3:  # Dừng khi đã có 3 thẻ <p>
                break
    # Đảo ngược danh sách nếu muốn giữ thứ tự từ gần đến xa so với tables[1]
    p_before_table1.reverse()
    # lấy text trong 3 thẻ p
    part_6_list = []
    for p in p_before_table1:
        text = p.get_text().replace('\r', ' ').replace('\n', ' ').replace('  ', ' ').strip()
        if "Căn cứ Thông tư số" not in text:
            part_6_list.append(text)
            part_6_text = '\n'.join(part_6_list)

    info_list.append(part2_text)
    info_list.append(part3)
    info_list.append(part3_text)
    info_list.append(part_6_text)
    
    info_list = [re.sub(r'[ ]+', ' ', text).strip() for text in info_list]
    
    print(info_list)
    return info_list

In [3]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [4]:
with open('links_for_craw.txt', 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [5]:
len(all_links)

91

In [6]:
all_maso_info_list = []
for link in all_links:
    if 'truoc-ma' in link:
    # if link:
        try:
            info_list = get_ma_so_link_info(link)
            all_maso_info_list.append(info_list)
            # write_data_to_excel(maso_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_maso_info_list.append([link,"lỗi"])
            # write_data_to_excel(phanloai_sheet,info_list)

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6450-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-APTAMIL-PROFUTURA-CESARBIOTIK-639434.aspx

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6450-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-APTAMIL-PROFUTURA-CESARBIOTIK-639434.aspx', '6450/TB-TCHQ', '23-12-2024', 'Tên thương mại: APTAMIL PROFUTURA CESARBIOTIK 2 FOLLOW ON FORMULA', 'Tên gọi theo cấu tạo, công dụng: Sản phẩm dinh dưỡng công thức APTAMIL PROFUTURA CESARBIOTIK 2 FOLLOW ON FORMULA (dành cho trẻ từ 12-24 tháng tuổi)', 'Ký, mã hiệu, chủng loại: không có', 'Nhà sản xuất: DANONE NUTRICIA NZ LTD', '2. Tóm tắt mô tả hàng hóa được xác định trước mã số: Theo hồ sơ xác định trước mã số, thông tin mặt hàng như sau:\n- Thành phần, cấu tạo, công thức hóa học, hàm lượng tính trên trọng lượng:\nSữa tách béo (sữa bò), Lactose (sữa bò), Sữa nguyên kem (sữa bò), Hỗn hợp dầu thực vật (dầu cọ olein, dầu hạt cải, dầu dừa, dầu hướng dương), bột đạm whey (sữa bò), Galacto-oligo

In [7]:
df = pd.DataFrame(all_maso_info_list)

In [ ]:
df.to_excel("truoc_ma_so.xlsx",index = False)